# Infrastructure Setup

Sets up Lakebase tables and syncs Delta data for the Spend Categorization app.

## Sections

1. **Spark Session** - Initialize Spark (Databricks or Databricks Connect)
2. **Configuration** - Load config.yaml (app and categorize settings)
3. **Backend** - Initialize database backend (Mock or Lakebase)
4. **Lakebase Tables** - Create reviews table
5. **Delta Sync** - Sync invoices from Delta to Lakebase
6. **Sample Data** - Optionally load test data
7. **Query Test** - Verify query functions
8. **Review Test** - Verify review functions

## Modes

- **test**: Uses in-memory mock data
- **prod**: Connects to Lakebase PostgreSQL


In [17]:
# Setup: Works both on Databricks and locally with Databricks Connect
from src.utils import get_spark, is_running_on_databricks

spark = get_spark()
print(f"✅ Spark session initialized")
print(f"   Running on Databricks: {is_running_on_databricks()}")
print(f"   Spark version: {spark.version}")


✅ Spark session initialized
   Running on Databricks: True
   Spark version: 4.0.0


In [18]:
from src.app.config import load_config
from src.utils import load_categorize_config

config = load_config()
cat_config = load_categorize_config()

print(f"Configuration Summary")
print(f"{'='*50}")
print(f"Mode:                    {config.mode}")
print(f"Invoices Sync Table:     {config.invoices_sync}")
print(f"Categorization Sync:     {config.categorization_sync}")
print(f"Reviews Table:           {config.reviews_table}")
print(f"Low Confidence Threshold: {config.low_confidence_threshold}")

if config.is_prod_mode:
    print(f"Lakebase Instance:       {config.lakebase_instance}")
    print(f"Lakebase Database:       {config.lakebase_dbname}")

print(f"\nDelta Tables (source):")
print(f"Catalog:                 {cat_config.catalog}")
print(f"Schema:                  {cat_config.schema_name}")
print(f"{'='*50}")


Configuration Summary
Mode:                    test
Invoices Sync Table:     invoices_sync
Categorization Sync:     categorization_sync
Reviews Table:           reviews
Low Confidence Threshold: 0.7

Delta Tables (source):
Catalog:                 shm
Schema:                  spend


## 3. Backend Initialization


In [19]:
from src.app.database import (
    init_backend,
    get_backend,
    reset_backend,
    MockBackend,
    LakebaseBackend,
)

reset_backend()
backend = init_backend(config)

print(f"Backend Initialization")
print(f"{'='*50}")
print(f"Backend Type:            {type(backend).__name__}")
print(f"Is Connected:            {backend.is_connected()}")

if isinstance(backend, MockBackend):
    print(f"Mock Invoices Count:     {len(backend._invoices)}")
    print(f"Mock Categories Count:   {len(backend._categories)}")
    print(f"Mock Reviews Count:      {len(backend._reviews)}")
else:
    if backend.is_connected():
        print(f"Connected to Lakebase: {config.lakebase_instance}")
    else:
        print(f"Failed to connect to Lakebase")


Backend Initialization
Backend Type:            MockBackend
Is Connected:            True
Mock Invoices Count:     100
Mock Categories Count:   15
Mock Reviews Count:      0


## 4. Lakebase Table Setup (Prod Mode Only)

Creates the reviews table in Lakebase. The invoices_sync and categorization_sync tables are created via sync in the next section.


In [20]:
from src.app.reviews import initialize_reviews_table

if config.is_prod_mode and isinstance(backend, LakebaseBackend):
    print("Initializing Lakebase Tables")
    print(f"{'='*50}")
    
    try:
        # Create reviews table (app writes here)
        print(f"Creating table: {config.reviews_table}...")
        initialize_reviews_table(config, backend)
        print(f"  Table {config.reviews_table} created/verified")
        
        print(f"\nReviews table ready")
    except Exception as e:
        print(f"Failed to initialize tables: {e}")
else:
    print("Skipping table initialization (test mode uses in-memory data)")


Skipping table initialization (test mode uses in-memory data)


## 5. Sync Delta Tables to Lakebase

Sync invoices and categorization data from Delta to Lakebase for low-latency app access.


In [21]:
def sync_delta_to_lakebase(
    spark,
    source_table: str,
    target_table: str,
    lakebase_instance: str,
    lakebase_dbname: str = "databricks_postgres",
):
    """
    Sync a Delta table to a Lakebase PostgreSQL table.
    
    Uses the Databricks SDK to get connection credentials and writes via JDBC.
    """
    from databricks.sdk import WorkspaceClient
    import uuid
    
    client = WorkspaceClient()
    
    # Get Lakebase connection info
    db_instance = client.database.get_database_instance(name=lakebase_instance)
    cred = client.database.generate_database_credential(
        request_id=str(uuid.uuid4()),
        instance_names=[lakebase_instance],
    )
    user = client.current_user.me()
    user_email = user.emails[0].value if user.emails else user.user_name
    
    # JDBC URL for Lakebase PostgreSQL
    jdbc_url = f"jdbc:postgresql://{db_instance.read_write_dns}:5432/{lakebase_dbname}?sslmode=require"
    
    # Read from Delta
    df = spark.table(source_table)
    row_count = df.count()
    print(f"Reading {row_count} rows from {source_table}")
    
    # Write to Lakebase via JDBC
    (
        df.write
        .format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", target_table)
        .option("user", user_email)
        .option("password", cred.token)
        .option("driver", "org.postgresql.Driver")
        .mode("overwrite")
        .save()
    )
    
    print(f"Synced {row_count} rows to {target_table}")
    return row_count


In [ ]:
# Sync invoices from Delta to Lakebase
# Set RUN_SYNC = True to execute

RUN_SYNC = False

if RUN_SYNC and config.is_prod_mode:
    print("Syncing Delta tables to Lakebase")
    print("=" * 50)
    
    # Source: silver.invoices -> Target: invoices_sync
    source_invoices = f"{cat_config.catalog}.{cat_config.schema_name}.invoices"
    
    sync_delta_to_lakebase(
        spark=spark,
        source_table=source_invoices,
        target_table=config.invoices_sync,
        lakebase_instance=config.lakebase_instance,
        lakebase_dbname=config.lakebase_dbname,
    )
    
    print("=" * 50)
    print("Sync complete")
else:
    print("Sync disabled. Set RUN_SYNC = True to sync Delta tables to Lakebase.")


## 6. Load Sample Data (Optional)

Load sample invoices directly to Lakebase for testing. Set `LOAD_SAMPLE_DATA = True` to enable.


In [ ]:
LOAD_SAMPLE_DATA = False
SAMPLE_COUNT = 100

if LOAD_SAMPLE_DATA and config.is_prod_mode:
    from src.utils import generate_sample_invoices
    
    print(f"Loading {SAMPLE_COUNT} Sample Invoices to {config.invoices_sync}")
    print("=" * 50)
    
    try:
        invoices_df = generate_sample_invoices(SAMPLE_COUNT, seed=42)
        
        # Add confidence column for categorization_sync simulation
        invoices_df["confidence"] = invoices_df["confidence_score"]
        invoices_df["run_id"] = "sample_run_001"
        
        for i, row in invoices_df.iterrows():
            insert_query = f"""
                INSERT INTO {config.invoices_sync}
                (invoice_id, invoice_number, transaction_id, vendor_name, 
                 invoice_date, amount, category, confidence_score, description)
                VALUES (:invoice_id, :invoice_number, :transaction_id, :vendor_name,
                        :invoice_date, :amount, :category, :confidence_score, :description)
                ON CONFLICT (invoice_id) DO NOTHING
            """
            backend.execute_write(insert_query, row.to_dict())
            
            if (i + 1) % 20 == 0:
                print(f"  Inserted {i + 1}/{SAMPLE_COUNT} invoices...")
        
        print(f"\nLoaded {SAMPLE_COUNT} sample invoices")
    except Exception as e:
        print(f"Failed to load sample data: {e}")
elif LOAD_SAMPLE_DATA:
    print("Skipping sample data load (test mode already has mock data)")
else:
    print("Sample data loading disabled. Set LOAD_SAMPLE_DATA = True to enable.")


⏭️ Sample data loading disabled. Set LOAD_SAMPLE_DATA = True to enable.


## 7. Query Functions Test

Test query functions with the backend.


In [ ]:
from src.app.queries import (
    search_invoices,
    get_flagged_invoices,
    get_invoices_by_ids,
    get_available_categories,
)

print("Testing Query Functions")
print("=" * 50)

# Test categories
categories = get_available_categories(config, backend)
print(f"Categories found: {len(categories)}")
if categories:
    print(f"Sample: {categories[:5]}")

# Test search
results = search_invoices(config, "bearing", limit=10, backend=backend)
print(f"Search 'bearing': {len(results)} results")

# Test invoices by ID
if isinstance(backend, MockBackend):
    test_ids = backend._invoices["invoice_id"].head(3).tolist()
    invoices = get_invoices_by_ids(config, test_ids, backend=backend)
    print(f"Get by IDs: {len(invoices)} invoices")

print("=" * 50)

[]

## 8. Review Functions Test

Test writing reviews to the reviews table.


In [ ]:
from src.app.reviews import write_review, write_reviews_batch

print("Testing Review Functions")
print("=" * 50)

if isinstance(backend, MockBackend):
    initial_count = len(backend._reviews)
    print(f"Initial reviews: {initial_count}")

# Write single review
write_review(
    config,
    invoice_id="TEST-INV-001",
    category="Hardware",
    schema_id="v1",
    prompt_id="p1",
    labeler_id="test_notebook",
    backend=backend,
)
print("Single review written")

# Write batch
batch_reviews = [
    {"invoice_id": "TEST-INV-002", "category": "Software", "schema_id": "v1", "prompt_id": "p1"},
    {"invoice_id": "TEST-INV-003", "category": "Cloud Services", "schema_id": "v1", "prompt_id": "p1"},
]
write_reviews_batch(config, batch_reviews, backend=backend)
print(f"Batch of {len(batch_reviews)} reviews written")

if isinstance(backend, MockBackend):
    final_count = len(backend._reviews)
    print(f"Total reviews: {final_count}")

print("=" * 50)


✏️ Testing Correction Functions

1. write_correction()
   Single correction written
   ✅ PASSED

2. write_corrections_batch()
   Batch of 2 corrections written
   ✅ PASSED

✅ All correction functions working correctly


## 9. Summary


In [ ]:
print("Infrastructure Setup Complete")
print("=" * 50)
print(f"Mode:              {config.mode}")
print(f"Backend:           {type(backend).__name__}")
print(f"Connected:         {backend.is_connected()}")
print()
print("Lakebase Tables:")
print(f"  invoices_sync:       {config.invoices_sync}")
print(f"  categorization_sync: {config.categorization_sync}")
print(f"  reviews:             {config.reviews_table}")
print()
print("Delta Tables (source):")
print(f"  {cat_config.catalog}.{cat_config.schema_name}.invoices")
print(f"  {cat_config.catalog}.{cat_config.schema_name}.cat_bootstrap")
print("=" * 50)


🔍 Testing Query Functions

1. get_available_categories()
   Categories found: 0
   Sample: []


AssertionError: Should have at least one category